In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [2]:
df = pd.read_csv("shampoo.csv")

In [3]:
df.head()

,Month,Sales
0,1-01,266.0
1,1-02,145.9
2,1-03,183.1
3,1-04,119.3
4,1-05,180.3


In [4]:
# tách lay thang 
df["month"] = [str(s.split("-")[0]) for s in df["Month"]]
# tach lay nam 
df["year"] = [int(s.split("-")[1]) for s in df["Month"]]
# tinh Năm
df["month"] = [str('190'+s) for s in df["month"]]
df.head()

,Month,Sales,month,year
0,1-01,266.0,1901,1
1,1-02,145.9,1901,2
2,1-03,183.1,1901,3
3,1-04,119.3,1901,4
4,1-05,180.3,1901,5


In [5]:
#Tạo cot từ Year và Month
df["Date"] = [str(y) + '-' + str(m) for y,m in zip(df["year"], df["month"])]
#chuyển đổi cột Date về dạng DateTime
df['Date']=pd.to_datetime(df["Date"])
df.head()

,Month,Sales,month,year,Date
0,1-01,266.0,1901,1,1901-01-01
1,1-02,145.9,1901,2,1901-02-01
2,1-03,183.1,1901,3,1901-03-01
3,1-04,119.3,1901,4,1901-04-01
4,1-05,180.3,1901,5,1901-05-01


In [6]:
df.drop(df.columns[[0,2,3]], axis=1, inplace=True)
df.set_index("Date",inplace=True)
df.head()

,Sales
Date,
1901-01-01,266.0
1901-02-01,145.9
1901-03-01,183.1
1901-04-01,119.3
1901-05-01,180.3


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 36 entries, 1901-01-01 to 1903-12-01
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Sales   36 non-null     float64
dtypes: float64(1)
memory usage: 576.0 bytes



## tạo tập train - test

In [8]:
# chuyen va dang value
data = df.values
# tap train la 18 dong dau , test là 18 dong sau
train = data[:-18]
test = data[-18:]

In [9]:
# Scale the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data) # hoc tap data de scaler ket qua 
len(scaled_data)

36

## tách tập train

In [10]:
# tạo vòng lặp giá trị với 6 ngày một vòng lặp
x_train,y_train=[],[]
for i in range(6,len(train)):
    # x nhan gia tri tu 1 den 6 và tang dan 2 den 7 ,... mot vong lap
    x_train.append(scaled_data[i-6:i,0]) # nhận giá trị trong khoảng từ i<-6 đơn vi
    # nhan gia tri của ngay tiep theo  làm du doan
    y_train.append(scaled_data[i,0])

In [11]:
# cchuyen dl ve kieu array
x_train1 = np.array(x_train)
y_train1 = np.array(y_train)

# dua dl ve mang 1 chieu 
x_train2 = np.reshape(x_train1, (x_train1.shape[0], x_train1.shape[1], 1))
y_train2 = np.reshape(y_train1, (y_train1.shape[0],1))

In [12]:
x_train2[:2]

array([[[0.2607073 ],
        [0.04727208],
        [0.11338191],
        [0.        ],
        [0.1084059 ],
        [0.08743558]],

       [[0.04727208],
        [0.11338191],
        [0.        ],
        [0.1084059 ],
        [0.08743558],
        [0.19992891]]])

In [13]:
y_train2[:2]

array([[0.19992891],
       [0.18695575]])

In [14]:
# import thư viện
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Bidirectional,Embedding
from keras.models import load_model, Model

In [15]:
# Xây dựng mạng nơ-ron đơn giản để tạo lớp mạng
model_lstm = Sequential()
#Xây dungje lớp mạng nơ-ron LSTM với số chiều là 64
model_lstm.add(LSTM(64,input_shape =(x_train2.shape[1],1),return_sequences=True))
# dropout trách việc học tủ của mạng 
model_lstm.add(Dropout(0.2))
# lớp out put đầr ra 1 chiều cho giá trị
model_lstm.add(Dense(1))
# Sai số toàn phương trung bình mse
model_lstm.compile(loss = 'mse')
model_lstm.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 6, 64)             16896     
                                                                 
 dropout (Dropout)           (None, 6, 64)             0         
                                                                 
 dense (Dense)               (None, 6, 1)              65        
                                                                 
Total params: 16,961
Trainable params: 16,961
Non-trainable params: 0
_________________________________________________________________


In [16]:

# mạng nơ- ron học 
model_lstm.fit(x_train2,y_train2,epochs = 10)

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 0.0399
Epoch 2/10
1/1 [==============================] - 0s 10ms/step - loss: 0.0315
Epoch 3/10
1/1 [==============================] - 0s 13ms/step - loss: 0.0266
Epoch 4/10
1/1 [==============================] - 0s 11ms/step - loss: 0.0230
Epoch 5/10
1/1 [==============================] - 0s 10ms/step - loss: 0.0206
Epoch 6/10
1/1 [==============================] - 0s 12ms/step - loss: 0.0185
Epoch 7/10
1/1 [==============================] - 0s 14ms/step - loss: 0.0175
Epoch 8/10
1/1 [==============================] - 0s 11ms/step - loss: 0.0167
Epoch 9/10
1/1 [==============================] - 0s 14ms/step - loss: 0.0153
Epoch 10/10
1/1 [==============================] - 0s 12ms/step - loss: 0.0154


In [17]:
# gia thuc
price = scaler.inverse_transform(y_train2)
price

array([[231.8],
       [224.5],
       [192.8],
       [122.9],
       [336.5],
       [185.9],
       [194.3],
       [149.5],
       [210.1],
       [273.3],
       [191.4],
       [287. ]])

In [18]:

# gia du doan 
predictions = model_lstm.predict(x_train2)
#predictions = scaler.inverse_transform(predictions)
predictions

1/1 [==============================] - 1s 736ms/step


array([[[0.05384691],
        [0.08880577],
        [0.12119298],
        [0.14603046],
        [0.16971904],
        [0.19113691]],

       [[0.04484933],
        [0.07661591],
        [0.10273989],
        [0.12874255],
        [0.15310642],
        [0.18047786]],

       [[0.04763599],
        [0.07641687],
        [0.10502101],
        [0.13188109],
        [0.16154926],
        [0.19168428]],

       [[0.04285732],
        [0.07309328],
        [0.10213223],
        [0.13418935],
        [0.16680224],
        [0.19606936]],

       [[0.04742622],
        [0.07980099],
        [0.11473759],
        [0.14985579],
        [0.1813106 ],
        [0.2036092 ]],

       [[0.04654222],
        [0.08312411],
        [0.12039258],
        [0.15426537],
        [0.17910236],
        [0.21401398]],

       [[0.05128473],
        [0.09050824],
        [0.12669232],
        [0.15401949],
        [0.19126455],
        [0.22254603]],

       [[0.05073779],
        [0.08717845],
        [0.1161598

## thực hiện test

In [19]:
#tạo vòng lặp giá trị với 6 ngày một vòng lặp
x_test,y_test = [],[]
for i in range(6,len(test)):
    # x nhan gia tri tu 1 den 6 và tang dan 2 den 7 ,... mot vong lap
    x_test.append(scaled_data[i-6:i])
    # nhan gia tri của ngay tiep theo  làm du doan
    y_test.append(scaled_data[i])
    
# cchuyen dl ve kieu array
x_test1 = np.array(x_test)
y_test1 = np.array(y_test)


# dua dl ve mang 1 chieu 
x_test2 = np.reshape(x_test1, (x_test1.shape[0], x_test1.shape[1], 1))
#y_test2 = np.reshape(y_test1, (y_test1.shape[0], y_test1.shape[1], 1))

In [20]:
x_test2[1]

array([[0.04727208],
       [0.11338191],
       [0.        ],
       [0.1084059 ],
       [0.08743558],
       [0.19992891]])

In [24]:
# du doan 
predictions = model_lstm.predict(x_test2)
# dung scaler để trả về ket qua 
#predictions = scaler.inverse_transform(predictions)
predictions

1/1 [==============================] - 0s 21ms/step


array([[[0.05384691],
        [0.08880577],
        [0.12119298],
        [0.14603046],
        [0.16971904],
        [0.19113691]],

       [[0.04484933],
        [0.07661591],
        [0.10273989],
        [0.12874255],
        [0.15310642],
        [0.18047786]],

       [[0.04763599],
        [0.07641687],
        [0.10502101],
        [0.13188109],
        [0.16154926],
        [0.19168428]],

       [[0.04285732],
        [0.07309328],
        [0.10213223],
        [0.13418935],
        [0.16680224],
        [0.19606936]],

       [[0.04742622],
        [0.07980099],
        [0.11473759],
        [0.14985579],
        [0.1813106 ],
        [0.2036092 ]],

       [[0.04654222],
        [0.08312411],
        [0.12039258],
        [0.15426537],
        [0.17910236],
        [0.21401398]],

       [[0.05128473],
        [0.09050824],
        [0.12669232],
        [0.15401949],
        [0.19126455],
        [0.22254603]],

       [[0.05073779],
        [0.08717845],
        [0.1161598